# 实现RNN   

In [1]:
import torch
import torch.nn as nn

In [3]:
rnn = nn.RNN(input_size = 10, hidden_size = 20, num_layers = 2)

In [7]:
# 第一层权重系数
print(rnn.weight_ih_l0.shape)
print(rnn.weight_hh_l0.shape)
print(rnn.bias_ih_l0.shape)

torch.Size([20, 10])
torch.Size([20, 20])
torch.Size([20])


In [9]:
# 第二层权重系数
print("win形状 {}".format(rnn.weight_ih_l1.shape))
print("whh形状 {}".format(rnn.weight_hh_l1.shape))
print("bih形状 {}".format(rnn.bias_ih_l1.shape))

win形状 torch.Size([20, 20])
whh形状 torch.Size([20, 20])
bih形状 torch.Size([20])


In [10]:
# 生成数据
# 输入特征长度100，批次32，特征维度10
input = torch.randn(100, 32, 10)
# 隐状态2层，32批次，维度20
h_0 = torch.randn(2, 32, 20)

In [11]:
output, h_n = rnn(input, h_0)
print(output.shape, h_n.shape)

torch.Size([100, 32, 20]) torch.Size([2, 32, 20])


In [12]:
# 使用pytorch构建RNN
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.i2o = nn.Linear(input_size+hidden_size, output_size)
        self.i2h = nn.Linear(input_size+hidden_size, hidden_size)
        self.softmax = nn.LogSoftmax(dim=1)
    
    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), dim = 1)
        output = self.i2o(combined)
        hidden = self.i2h(combined)
        output = self.softmax(output)
        return output, hidden
    
    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

In [13]:
n_hiddens = 128
rnn = RNN(10, n_hiddens, 10)

In [15]:
print(rnn)

RNN(
  (i2o): Linear(in_features=138, out_features=10, bias=True)
  (i2h): Linear(in_features=138, out_features=128, bias=True)
  (softmax): LogSoftmax(dim=1)
)


# 实现LSTM

In [1]:
import torch
import torch.nn as nn

In [17]:
lstm = nn.LSTM(input_size = 10, hidden_size = 20, num_layers = 2)

In [18]:
print(lstm)

LSTM(10, 20, num_layers=2)


In [20]:
print("wih形状 {} ".format(lstm.weight_ih_l0.shape))
print("whh形状 {} ".format(lstm.weight_hh_l0.shape))
print("bih形状 {} ".format(lstm.bias_ih_l0.shape))

wih形状 torch.Size([80, 10]) 
whh形状 torch.Size([80, 20]) 
bih形状 torch.Size([80]) 


In [21]:
print("wih形状 {} ".format(lstm.weight_ih_l1.shape))
print("whh形状 {} ".format(lstm.weight_hh_l1.shape))
print("bih形状 {} ".format(lstm.bias_ih_l1.shape))

wih形状 torch.Size([80, 20]) 
whh形状 torch.Size([80, 20]) 
bih形状 torch.Size([80]) 


In [24]:
# 生成数据
# 输入特征长度100，批次32，特征维度10
input = torch.randn(100, 32, 10)
# 隐状态2层，32批次，维度20
h_0 = torch.randn(2, 32, 20)
h0 = (h_0, h_0)

In [26]:
output, h_n = lstm(input, h0)
print(output.size(), h_n[0].size(), h_n[1].size())

torch.Size([100, 32, 20]) torch.Size([2, 32, 20]) torch.Size([2, 32, 20])


In [31]:
# 测试linear
liear = nn.Linear(10, 5)
print(liear.weight.shape)

torch.Size([5, 10])


In [3]:
# 使用pytorch实现LSTM
class LSTMCell(nn.Module):
    def __init__(self, input_size, hidden_size, cell_size, output_size):
        super(LSTMCell, self).__init__()
        self.hidden_size = hidden_size
        self.cell_size = cell_size
        self.gate = nn.Linear(input_size+hidden_size, cell_size)
        self.output = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()
        self.tanh = nn.Tanh()
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden, cell):
        combined = torch.cat((input, hidden), dim=1)
        f_gate = self.sigmoid(self.gate(combined))  # 遗忘门
        o_gate = self.sigmoid(self.gate(combined))  # 输出门
        i_gate = self.sigmoid(self.gate(combined))  # 输入门
        c_state = self.tanh(self.gate(combined))     # 局部留存
        cell = torch.add(torch.mul(cell, f_gate), torch.mul(i_gate, c_state))   # 内部状态
        hidden = torch.mul(o_gate, self.tanh(cell))    # 隐状态
        output = self.output(hidden)
        output = self.softmax(output)
        return output, hidden, cell
    
    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

    def initCell(self):
        return torch.zeros(1, self.cell_size) 


In [4]:
lstmcell = LSTMCell(input_size=10, hidden_size=20, cell_size=20, output_size=10)

In [7]:
input = torch.randn(32, 10)
h_0 = torch.randn(32, 20)

In [8]:
output, h_n, c_n = lstmcell(input, h_0, h_0)
print(output.size(), h_n.size(), c_n.size())

torch.Size([32, 10]) torch.Size([32, 20]) torch.Size([32, 20])


# 实现GRU

In [12]:
class GRUCell(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(GRUCell, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.gate = nn.Linear(input_size+hidden_size, hidden_size)
        self.output = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()
        self.tanh = nn.Tanh()
        self.softmax = nn.LogSoftmax(dim = 1)

    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), dim=1)
        z_gate = self.sigmoid(self.gate(combined)) # 更新门
        r_gate = self.sigmoid(self.gate(combined)) # 重置门
        combined01 = torch.cat((input, torch.mul(hidden, r_gate)), dim=1)
        h1_state = self.tanh(self.gate(combined01)) # 候选状态
        h_state = torch.add(torch.mul(z_gate, h1_state), torch.mul((1-z_gate), hidden))
        output = self.softmax(self.output(h_state))
        return output, h_state
    
    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

In [13]:
grucell = GRUCell(input_size=10, hidden_size=20, output_size=10)

In [14]:
input = torch.randn(32, 10)
h_0 = torch.randn(32, 20)

In [15]:
output, h_n = grucell(input, h_0)
print(output.size(), h_n.size())

torch.Size([32, 10]) torch.Size([32, 20])
